FOR SIX Language as we discussed we are going to implement English,Hindi,gujrati,punjabi,bengali,tamil

In [140]:
import numpy as np
import nltk
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ujjwalgoel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [141]:
paras_e = open('/Users/ujjwalgoel/Downloads/IR_Project_15-main/15_MidProject/Dataset/english.txt', encoding="UTF8").read().split("\n")
paras_h = open('/Users/ujjwalgoel/Downloads/IR_Project_15-main/15_MidProject/Dataset/hindi.txt', encoding="UTF8").read().split("\n")
paras_g = open('/Users/ujjwalgoel/Downloads/IR_Project_15-main/15_MidProject/Dataset/gujarati.txt', encoding="UTF8").read().split("\n")
paras_p = open('/Users/ujjwalgoel/Downloads/IR_Project_15-main/15_MidProject/Dataset/punjabi.txt', encoding="UTF8").read().split("\n")
paras_t = open('/Users/ujjwalgoel/Downloads/IR_Project_15-main/15_MidProject/Dataset/tamil.txt', encoding="UTF8").read().split("\n")
paras_b = open('/Users/ujjwalgoel/Downloads/IR_Project_15-main/15_MidProject/Dataset/bengali.txt', encoding="UTF8").read().split("\n")


Stemming and tokensing Hindi words

In [63]:
suffixes = {
    1: ["ो", "े", "ू", "ु", "ी", "ि", "ा"],
    2: ["कर", "ाओ", "िए", "ाई", "ाए", "ने", "नी", "ना", "ते", "ीं", "ती", "ता", "ाँ", "ां", "ों", "ें"],
    3: ["ाकर", "ाइए", "ाईं", "ाया", "ेगी", "ेगा", "ोगी", "ोगे", "ाने", "ाना", "ाते", "ाती", "ाता", "तीं", "ाओं", "ाएं", "ुओं", "ुएं", "ुआं"],
    4: ["ाएगी", "ाएगा", "ाओगी", "ाओगे", "एंगी", "ेंगी", "एंगे", "ेंगे", "ूंगी", "ूंगा", "ातीं", "नाओं", "नाएं", "ताओं", "ताएं", "ियाँ", "ियों", "ियां"],
    5: ["ाएंगी", "ाएंगे", "ाऊंगी", "ाऊंगा", "ाइयाँ", "ाइयों", "ाइयां"],
}

def stemming_hindi(text):
    for L in 5, 4, 3, 2, 1:
        if len(text) > L + 1:
            for suf in suffixes[L]:
                if text.endswith(suf):
                    return text[:-L]
    return text

In [160]:
import string as st

def preprocess_text(string, stopWords):
    # Define a set of punctuation marks to remove
    punctuations = set(st.punctuation) | set(["०", "S", "―", "=", "॥", "”", "|", "“", "'", "।", ";", ",", ":", "!", '"', "?", "[", "]", "......", ":-", ".", ":-", "{", "(", "}", ")"])
    
    # Remove punctuations and split the string into tokens
    tokens = string.translate(str.maketrans("", "", "".join(punctuations))).split()
    
    # Apply stemming and remove stop words from the tokens
    tokens = [stemming_hindi(word) for word in tokens if word not in stopWords]
    
    return tokens

In [161]:
pos_lis = {}
for idx, para in enumerate(paras_h):
    words = preprocess_text(para, stopWords)
    
    for word in words:
        if word in pos_lis.keys():
            if idx in pos_lis[word].keys():
                pos_lis[word][idx] += 1
            else:
                pos_lis[word][idx] = 1
        else:
            temp = {idx : 1}
            pos_lis[word] = temp.copy()

In [162]:
print(len(pos_lis))

2253


BM25

In [163]:
import numpy as np
from collections import defaultdict


def bm25(query, paragraphs, pos_lisl, stop_words, l, b=0.75, k=2):
    q_tokens = preprocess_text(query, stop_words)
    lengths = [len(p.split()) for p in paragraphs]
    N = len(paragraphs)
    avg_len = sum(lengths) / N

    idf = {}
    for word in np.unique(q_tokens):
        if word in pos_lisl:
            df = len(pos_lisl[word])
        else:
            df = 0
        idf[word] = np.log((N - df + 0.5) / (df + 0.5))

    score = defaultdict(float)
    for idx, para in enumerate(paragraphs):
        para_tokens = para.split()
        for word in q_tokens:
            tf = para_tokens.count(word)
            if word in pos_lisl and idx in pos_lisl[word]:
                tf = pos_lisl[word][idx]
            score[idx] += idf[word] * (tf * (k + 1)) / (k * (1 - b + b * lengths[idx] / avg_len) + tf)

    return sorted(score, key=score.get, reverse=True)[:l]


Searching and Evaluating

In [164]:
def search(query, pos_lis, lang, l):
    # print(query)
    # print(pos_lis)
    query = get_translation(query, "hi")
    # print(query)
    # print(query)
    tokens = preprocess_text(query, stopWords)
   
    query = " ".join(tokens)

    indxs = bm25(query, paras_e,pos_lis,stopWords, l,0.75,2)
    # print(indxs)
    if lang == 'e':
        return [f'{paras_e[idx]}' for idx in indxs], indxs
    elif lang== 'g':
        return [f'{paras_g[idx]}' for idx in indxs], indxs

    elif lang=='h':
        return [f'{paras_h[idx]}' for idx in indxs], indxs
    elif lang=='b':
        return [f'{paras_b[idx]}' for idx in indxs], indxs
    elif lang=='t':
        return [f'{paras_t[idx]}' for idx in indxs], indxs
    elif lang=='p':
        return [f'{paras_p[idx]}' for idx in indxs], indxs

In [146]:
from googletrans import Translator

def get_translation(data, dest):
    translator = Translator()
    text = translator.translate(data, dest).text
    return text

In [165]:
def print_shloka(shloka):
    n =len(shloka.split())
    shloka = shloka.split()
    idx1 = n//4
    idx2 = n//2
    idx3 = (3*n)//4

    print(" ".join(shloka[:idx1]))
    print(" ".join(shloka[idx1:idx2]))
    print(" ".join(shloka[idx2:idx3]))
    print(" ".join(shloka[idx3:]))

In [191]:
query ="praise shame evil lust"
result = search(query,pos_lis,"p",5)

print("The top 5 relevant documents for the query \" ",query," \" in the asked language are : ")
for i in range(5):
    print("\nshloka ",i+1," (index = ",result[1][i]," ) =>  \n")

    print_shloka(result[0][i])

The top 5 relevant documents for the query "  praise shame evil lust  " in the asked language are : 

shloka  1  (index =  355  ) =>  

ਜੋ ਲੋਕ ਮੈਨੂੰ ਅਜੰਮੇ, ਅਨਾਦਿ ਅਤੇ ਸਾਰੇ
ਬ੍ਰਹਿਮੰਡਾਂ ਦੇ ਸੁਆਮੀ ਦੇ ਰੂਪ ਵਿੱਚ ਜਾਣਦੇ ਹਨ,
ਕੇਵਲ ਉਹ ਮਨੁੱਖਾਂ ਵਿੱਚ ਭੁਲੇਖੇ ਤੋਂ ਮੁਕਤ
ਅਤੇ ਸਾਰੀਆਂ ਬੁਰਾਈਆਂ ਤੋਂ ਮੁਕਤ ਹੋ ਜਾਂਦੇ ਹਨ।

shloka  2  (index =  362  ) =>  

ਉਨ੍ਹਾਂ ਦੀ ਦਇਆ ਦੇ ਕਾਰਨ, ਮੈਂ ਉਨ੍ਹਾਂ
ਦੇ ਹਿਰਦੇ ਵਿੱਚ ਵਾਸ ਕਰਦਾ ਹਾਂ ਅਤੇ
ਗਿਆਨ ਦੇ ਪ੍ਰਕਾਸ਼ ਦੀਵੇ ਨਾਲ ਅੰਧਕਾਰ ਤੋਂ
ਪੈਦਾ ਹੋਈ ਅਗਿਆਨਤਾ ਦਾ ਨਾਸ ਕਰਦਾ ਹਾਂ।

shloka  3  (index =  628  ) =>  

ਕਿਸੇ ਨੂੰ ਆਪਣੇ ਸੁਭਾਅ ਤੋਂ ਪੈਦਾ ਹੋਏ ਕਰਤੱਵਾਂ ਨੂੰ ਨਹੀਂ ਛੱਡਣਾ
ਚਾਹੀਦਾ, ਭਾਵੇਂ ਕੋਈ ਉਨ੍ਹਾਂ ਵਿੱਚ ਨੁਕਸ ਦੇਖਦਾ ਹੋਵੇ। ਹੇ ਕੁੰਤੀ ਦੇ
ਪੁੱਤਰ! ਅਸਲ ਵਿੱਚ ਹਰ ਕਿਸਮ ਦੇ ਉਦਯੋਗ ਕਿਸੇ ਨਾ ਕਿਸੇ ਬੁਰਾਈ
ਨਾਲ ਢੱਕੇ ਹੋਏ ਹਨ ਜਿਵੇਂ ਅੱਗ ਧੂੰਏਂ ਨਾਲ ਢੱਕੀ ਹੋਈ ਹੈ।

shloka  4  (index =  544  ) =>  

ਸੈਕੜੇ ਇੱਛਾਵਾਂ ਦੇ ਬੰਧਨ ਵਿੱਚ ਰਹਿ ਕੇ, ਕਾਮ
ਕ੍ਰੋਧ ਤੋਂ ਪ੍ਰੇਰਿਤ ਹੋ ਕੇ, ਉਹ ਨਜਾਇਜ਼ ਧਨ
ਇਕੱਠਾ ਕਰਨ ਵਿੱਚ ਲੱਗੇ ਰਹਿੰਦੇ ਹਨ। ਉਹ ਇਹ
ਸਭ ਕੁਝ ਇੰਦਰੀਆਂ ਦੀ ਤ੍ਰਿਪਤੀ ਲਈ ਕਰਦੇ ਹਨ।

shloka  5  (index =  142  ) =>  

ਪਰਮ ਆਤਮਾ ਸ਼੍ਰੀ ਕ੍ਰਿਸ਼ਨ ਜੀ ਕਹਿੰਦੇ ਹਨ - ਕੇਵਲ ਵਾਸਨਾ
ਜੋ ਰਜੋਗੁਣ ਦੇ ਸੰਪਰ

In [192]:
query ="Actions based upon sacrifice and charity"
result = search(query,pos_lis,"g",5)

print("The top 5 relevant documents for the query \" ",query," \" in the asked language are : ")
for i in range(5):
    print("\nshloka ",i+1," (index = ",result[1][i]," ) =>  \n")

    print_shloka(result[0][i])

The top 5 relevant documents for the query "  Actions based upon sacrifice and charity  " in the asked language are : 

shloka  1  (index =  586  ) =>  

યજ્ઞ, દાન અને તપ પર આધારિત
કર્મોનો ક્યારેય ત્યાગ ન કરવો જોઈએ. તેઓ
ચોક્કસપણે થવું જોઈએ. યજ્ઞ, દાન અને તપ
ખરેખર મહાન આત્માઓને પણ શુદ્ધ કરે છે.

shloka  2  (index =  584  ) =>  

કેટલાક ઋષિમુનિઓ જાહેર કરે છે કે તમામ પ્રકારના
કાર્યોને દોષપૂર્ણ માનીને છોડી દેવા જોઈએ, પરંતુ અન્ય
વિદ્વાનો ભારપૂર્વક કહે છે કે યજ્ઞ, દાન અને
તપસ્યા જેવા કાર્યોને ક્યારેય છોડી દેવા જોઈએ નહીં.

shloka  3  (index =  566  ) =>  

શ્રદ્ધા વિના અને શાસ્ત્રોની આજ્ઞાથી વિપરિત યજ્ઞ,
જેમાં અન્ન અર્પણ કરવામાં આવતું નથી, મંત્રોનો
જાપ કરવામાં આવતો નથી અને દાન આપવામાં
આવતું નથી, આવો યજ્ઞ સ્વભાવે તમોગુણી છે.

shloka  4  (index =  575  ) =>  

આવા દાન જે અપવિત્ર સ્થાને અને
અયોગ્ય સમયે અયોગ્ય વ્યક્તિઓને અથવા આદર વિના
અથવા તિરસ્કાર સાથે આપવામાં આવે છે,
તે તમોગુણી સ્વભાવનું દાન માનવામાં આવે છે.

shloka  5  (index =  577  ) =>  

તેથી જ વેદના ઘાતાક દ્વારા
યજ્ઞ, દાન અને તપસ્યા જેવા 

In [193]:
query ="Actions based upon sacrifice and charity"
result = search(query,pos_lis,"e",5)

print("The top 5 relevant documents for the query \" ",query," \" in the asked language are : ")
for i in range(5):
    print("\nshloka ",i+1," (index = ",result[1][i]," ) =>  \n")

    print_shloka(result[0][i])

The top 5 relevant documents for the query "  Actions based upon sacrifice and charity  " in the asked language are : 

shloka  1  (index =  586  ) =>  

Actions based upon sacrifice, charity, and penance
should never be abandoned; they must certainly be
performed. Indeed, acts of sacrifice, charity, and penance
are purifying even for those who are wise

shloka  2  (index =  584  ) =>  

Some learned people declare that all kinds
of actions should be given up as
evil, while others maintain that acts of
sacrifice, charity, and penance should never be abandoned

shloka  3  (index =  566  ) =>  

Sacrifice devoid of faith and contrary to the
injunctions of the scriptures, in which no food is
offered, no mantras chanted, and no donation made,
is to be considered in the mode of ignorance

shloka  4  (index =  575  ) =>  

And that charity, which is given at
the wrong place and wrong time to unworthy
persons, without showing respect, or with contempt, is
held to be of the nature of nescience

TF-IDF model 

In [168]:

from collections import defaultdict
import numpy as np

def calculate_lengths(paras_h):
    lengths = defaultdict(int)
    for idx, para in enumerate(paras_h, start=1):
        lengths[idx] = len(para.split())
    return lengths

def calculate_idf(pos_lis, N):
    idf = {}
    for word in pos_lis.keys():
        df = len(pos_lis[word].keys())
        idf[word] = np.log((N) / (df))
    return idf

def calculate_norm_list(paras_h, idf, pos_lis, lengths):
    norm_list = defaultdict(list)
    for idx, para in enumerate(paras_h, start=1):
        for word in np.unique(para.split()):
            tf = 0
            if word in idf:
                try:
                    if idx in pos_lis[word].keys():
                        tf = pos_lis[word][idx] / lengths[idx]
                except KeyError:
                    pass
                norm_list[idx].append(tf * idf[word])
    return norm_list

def calculate_norms(N, norm_list):
    for idx in range(1, N+1):
        norm_list[idx] = np.linalg.norm(norm_list[idx])
    return norm_list

def calculate_tfidf(paras_h, pos_lis):
    lengths = calculate_lengths(paras_h)
    N = len(paras_h)
    idf = calculate_idf(pos_lis, N)
    norm_list = calculate_norm_list(paras_h, idf, pos_lis, lengths)
    norm_list = calculate_norms(N, norm_list)
    return norm_list


In [169]:
tfidf_scores = calculate_tfidf(paras_h, pos_lis)

In [117]:
def find_cosine_sim(n_docs, query_vector, doc_vectors):
    cosine_sims = []
    for i in range(n_docs-1):
        dot = np.dot(query_vector, doc_vectors[i])
        query_norm = np.linalg.norm(query_vector)
        doc_norm = norm_list[i]
        cosine_sims.append(dot/((query_norm + 0.5)*(doc_norm + 0.5)))
    return cosine_sims

In [171]:
def tfidf(query, main_dict,l):
    n_slokas = len(paras_h)
    tfidf_dict = {} # dictionary which stores tfidf of each document
    doc_vectors = [] 
    for i in range(n_slokas):
        vector = []
        for word in set(query):
            if word in main_dict and i in main_dict[word]:
                tf = main_dict[word][i]
                idf = np.log((n_slokas+1)/(len(main_dict[word])+1))
                tf_idf = tf*idf
                vector.append(tf_idf)
            else:
                vector.append(0)
        doc_vectors.append(vector)
    query_vector = []
    for word in set(query):
        if word in main_dict:
            tf = query.count(word)
            idf = np.log((n_slokas+1)/(len(main_dict[word])+1))
            tf_idf = tf*idf
            query_vector.append(tf_idf)
        else:
            query_vector.append(0)
    scores = find_cosine_sim(n_slokas, query_vector, doc_vectors)
    return sorted(np.argsort(np.array(scores))[:l], reverse=True)


In [172]:
def search(query, pos_lis, lang, l):
    query = get_translation(query, "hi")
    tokens = preprocess_text(query, stopWords)
    
    query = " ".join(tokens)

    indxs = bm25(query, pos_lis, l)
    # print(indxs)
    if lang == 'e':
        return [f'{paras_e[idx]}' for idx in indxs], indxs
    elif lang== 'g':
        return [f'{paras_g[idx]}' for idx in indxs], indxs

    elif lang=='h':
        return [f'{paras_h[idx]}' for idx in indxs], indxs
    elif lang=='b':
        return [f'{paras_b[idx]}' for idx in indxs], indxs
    elif lang=='t':
        return [f'{paras_t[idx]}' for idx in indxs], indxs
    elif lang=='p':
        return [f'{paras_p[idx]}' for idx in indxs], indxs

In [80]:
queries =[]
relevant = []

In [195]:
query ="જબરદસ્ત અવાજ આખા આકાશ અને પૃથ્વી પર ગર્જ્યો"
result = search(query,pos_lis,"e",l=5)
queries.append(query)
relevant.append(result[1])
print("The top 5 relevant documents using TF_IDF model for the query \" ",query," \" in the asked language are : ")
for i in range(5):
    print("\nshloka ",i+1," (index = ",result[1][i]," ) =>  \n")

    print_shloka(result[0][i])

The top 5 relevant documents using TF_IDF model for the query "  જબરદસ્ત અવાજ આખા આકાશ અને પૃથ્વી પર ગર્જ્યો  " in the asked language are : 

shloka  1  (index =  14  ) =>  

The terrific sound thundered
across the sky and the
earth, and shattered the hearts
of your sons, O Dhritarasthra

shloka  2  (index =  268  ) =>  

Earth, water, fire, air,
space, mind, intellect, and
ego—these are eight components
of My material energy

shloka  3  (index =  403  ) =>  

If a thousand suns were
to blaze forth together in the
sky, they would not match
the splendor of that great form

shloka  4  (index =  411  ) =>  

The space between heaven and earth and all
the directions is pervaded by You alone. Seeing Your
wondrous and terrible form, I see the three
worlds trembling in fear, O Greatest of all beings

shloka  5  (index =  72  ) =>  

If you fight, you will either be slain on the
battlefield and go to the celestial abodes, or you will
gain victory and enjoy the kingdom on earth. Therefore arise

In [196]:
query ="என் உடம்பெல்லாம் நடுங்குகிறது; என் தலைமுடி உதிர்ந்து நிற்கிறது"
result = search(query,pos_lis,"h",l=5)
queries.append(query)
relevant.append(result[1])
print("The top 5 relevant documents using TF_IDF model for the query \" ",query," \" in the asked language are : ")
for i in range(5):
    print("\nshloka ",i+1," (index = ",result[1][i]," ) =>  \n")

    print_shloka(result[0][i])

The top 5 relevant documents using TF_IDF model for the query "  என் உடம்பெல்லாம் நடுங்குகிறது; என் தலைமுடி உதிர்ந்து நிற்கிறது  " in the asked language are : 

shloka  1  (index =  23  ) =>  

मेरा सारा शरीर काँप रहा है, मेरे शरीर के रोएं खड़े हो रहे हैं, मेरा धनुष ‘गाण्डीव' मेरे हाथ से सरक रहा है और
मेरी पूरी त्वचा में जलन हो रही है। मेरा मन उलझ रहा है और मुझे घबराहट हो रही है। अब मैं यहाँ और अधिक
खड़ा रहने में समर्थ नहीं हूँ। केशी राक्षस को मारने वाले हे केशव! मुझे केवल अमंगल के लक्षण दिखाई दे रहे हैं। युद्ध में
अपने वंश के बंधु बान्धवों का वध करने में मुझे कोई अच्छाई नही दिखाई देती है और उन्हें मारकर मैं कैसे सुख पा सकता हूँ?

shloka  2  (index =  489  ) =>  

अंतरिक्ष सबको अपने में धारण कर लेता है लेकिन सूक्ष्म होने के
कारण जिसे यह धारण किए रहता है उसमें लिप्त नहीं होता। इसी
प्रकार से यद्यपि आत्मा चेतना के रूप में पूरे शरीर में व्याप्त
रहती है फिर भी आत्मा शरीर के धर्म से प्रभावित नहीं होती।

shloka  3  (index =  339  ) =>  

किन्तु जो लोग सदैव मेरे बारे में सोचते हैं और मेरी
अनन्

In [201]:
query ="qualities and activities"
result = search(query,pos_lis,"e",l=10)
queries.append(query)
relevant.append(result[1])
print("The top 20 relevant documents for the query \" ",query," \" in the asked language are : ")
for i in range(10):
    print("\nshloka ",i+1," (index = ",result[1][i]," ) =>  \n")

    print_shloka(result[0][i])

The top 20 relevant documents for the query "  qualities and activities  " in the asked language are : 

shloka  1  (index =  537  ) =>  

The divine qualities lead to liberation, while
the demoniac qualities are the cause for a
continuing destiny of bondage. Grieve not, O
Arjun, as you were born with saintly virtues

shloka  2  (index =  293  ) =>  

Those who take shelter in Me, striving
for liberation from old-age and death, come
to know the Brahman, the individual self,
and the entire field of karmic action

shloka  3  (index =  31  ) =>  

Through the evil deeds of those who
destroy the family tradition and thus give
rise to unwanted progeny, a variety of
social and family welfare activities are ruined

shloka  4  (index =  509  ) =>  

Arjun inquired: What are the characteristics of
those who have gone beyond the three guṇas,
O Lord? How do they act? How do
they go beyond the bondage of the guṇas

shloka  5  (index =  621  ) =>  

The duties of the Brahmins,
Kshatriyas, Vaishyas,

EVALUATION OF BM25 AND TF-IDF MODEL

In [202]:
def evaluate_models(documents, queries, relevant_docs, model_type):
    if model_type == 'tfidf':
        vectorizer = TfidfVectorizer()
        doc_vectors = vectorizer.fit_transform(documents)
        query_vectors = vectorizer.transform(queries)
        similarities = cosine_similarity(query_vectors, doc_vectors)
    elif model_type == 'bm25':
        tokenized_docs = [doc.split() for doc in documents]
        bm25 = BM25Okapi(tokenized_docs)
        similarities = [bm25.get_scores(query.split()) for query in queries]
    else:
        raise ValueError('Model type must be "bm25" or "tfidf"')
    
    num_queries = len(queries)
    avg_precision = 0
    avg_recall = 0
    avg_f1 = 0
    mean_average_precision = 0
    
    for i in range(num_queries):
        retrieved_docs = sorted(range(len(similarities[i])), key=lambda k: similarities[i][k], reverse=True)
        # print(retrieved_docs)
     
        relevant = set(relevant_docs[i])
        retrieved = set(retrieved_docs[:len(relevant_docs[i])])
        true_positives = relevant.intersection(retrieved)
        # print(true_positives)
        false_positives = retrieved - relevant 
        false_negatives = relevant - retrieved 
        # print(false_positives)
        # print(false_negatives)
        len_true_positives = len(true_positives)+1
        len_false_positives = len(false_positives)+1
        len_false_negatives = len(false_negatives)+1
        precision = len_true_positives / (len_true_positives + len_false_positives)
        recall = len_true_positives / (len_true_positives + len_false_negatives)
        f1 = 2 * (precision * recall) / (precision + recall)
        
        avg_precision += precision
        avg_recall += recall
        avg_f1 += f1

        ap = 0
        num_relevant = len(relevant)
        for j in range(num_relevant):
            if retrieved_docs[j] in relevant:
                ap += len(relevant.intersection(set(retrieved_docs[:j+1]))) / (j+1)
        ap /= num_relevant

        mean_average_precision += ap
    
    avg_precision /= num_queries
    avg_recall /= num_queries
    avg_f1 /= num_queries
    mean_average_precision /= num_queries
    
    return avg_precision, avg_recall, avg_f1,mean_average_precision


In [207]:
a,b,c,d = evaluate_models(paras_e,queries,relevant,"tfidf")

In [208]:
print("precision = ", a)
print("recall = ", b)
print("f1 = ", c)
print("map = ", d)

precision =  0.336038961038961
recall =  0.336038961038961
f1 =  0.336038961038961
map =  0.22477580404631492


In [209]:
a,b,c,d = evaluate_models(paras_e,queries,relevant,"bm25")

In [210]:
print("precision = ", a)
print("recall = ", b)
print("f1 = ", c)
print("map = ", d)

precision =  0.3147546897546898
recall =  0.3147546897546898
f1 =  0.3147546897546898
map =  0.1991463793988949
